In [1]:
import pandas as pd
import numpy as np
from scipy.special import softmax

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer

import csv
import urllib.request

In [2]:
df_1 = pd.read_csv('/kaggle/input/masculinity-saturday-twitter-sentiment-analysis/twitter_sentiment_analysis_masculinity_saturday_data_annotated.csv', index_col=0)

In [3]:
df_1.head()

,tweet_created_at,text,Positive,Negative,Neutral,Compound,Sentiment
0,2022-12-30 22:37:46+00:00,Men who have refused to listen to/attend @amer...,0.000,0.073,0.927,-0.2960,Negative
1,2022-12-30 22:30:58+00:00,@OvOBrezzzy Take amerix next,0.000,0.000,1.000,0.0000,Neutral
2,2022-12-30 22:18:00+00:00,@AyodeleYo11 @amerix Nah I’m good my masculini...,0.493,0.068,0.439,0.8271,Positive
3,2022-12-30 20:42:54+00:00,@Amerix_DontSimp There is no need for dating i...,0.000,0.180,0.820,-0.2960,Negative
4,2022-12-30 20:32:23+00:00,@DyeAnna7 @amerix @kibe From what cos nothing ...,0.110,0.000,0.890,0.6908,Positive


In [4]:
df_1.columns

Index(['tweet_created_at', 'text', 'Positive', 'Negative', 'Neutral',
       'Compound', 'Sentiment'],
      dtype='object')

In [5]:
df = df_1[['tweet_created_at', 'text', 'Sentiment']]

In [6]:
df.head()

,tweet_created_at,text,Sentiment
0,2022-12-30 22:37:46+00:00,Men who have refused to listen to/attend @amer...,Negative
1,2022-12-30 22:30:58+00:00,@OvOBrezzzy Take amerix next,Neutral
2,2022-12-30 22:18:00+00:00,@AyodeleYo11 @amerix Nah I’m good my masculini...,Positive
3,2022-12-30 20:42:54+00:00,@Amerix_DontSimp There is no need for dating i...,Negative
4,2022-12-30 20:32:23+00:00,@DyeAnna7 @amerix @kibe From what cos nothing ...,Positive


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248738 entries, 0 to 248738
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   tweet_created_at  248738 non-null  object
 1   text              248738 non-null  object
 2   Sentiment         248738 non-null  object
dtypes: object(3)
memory usage: 7.6+ MB


In [8]:
df['tweet_created_at'] = pd.to_datetime(df['tweet_created_at'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
df['Sentiment'] = df['Sentiment'].map({'Negative':0,'Neutral':1, 'Positive':2})

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
df.head()

,tweet_created_at,text,Sentiment
0,2022-12-30 22:37:46+00:00,Men who have refused to listen to/attend @amer...,0
1,2022-12-30 22:30:58+00:00,@OvOBrezzzy Take amerix next,1
2,2022-12-30 22:18:00+00:00,@AyodeleYo11 @amerix Nah I’m good my masculini...,2
3,2022-12-30 20:42:54+00:00,@Amerix_DontSimp There is no need for dating i...,0
4,2022-12-30 20:32:23+00:00,@DyeAnna7 @amerix @kibe From what cos nothing ...,2


In [11]:
def preprocess(text):
    
    new_text = []
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        
        new_text.append(t)
        
    return " ".join(new_text)

In [12]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

In [13]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [14]:
labels=[]

mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"

with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
    
labels = [row[1] for row in csvreader if len(row) > 1]
